In [1]:
import pandas as pd
import numpy as np

In [23]:
import matplotlib.pyplot as plt

In [2]:
admit_raw = pd.read_csv("Admission_Predict_Ver1.1.csv")
admit_x = admit_raw[['GRE Score', 'TOEFL Score', 'University Rating', 'SOP', 'LOR ', 'CGPA', 'Research']]
admit_y = admit_raw['Chance of Admit ']

In [3]:
print(admit_x.head())

   GRE Score  TOEFL Score  University Rating  SOP  LOR   CGPA  Research
0        337          118                  4  4.5   4.5  9.65         1
1        324          107                  4  4.0   4.5  8.87         1
2        316          104                  3  3.0   3.5  8.00         1
3        322          110                  3  3.5   2.5  8.67         1
4        314          103                  2  2.0   3.0  8.21         0


In [4]:
from sklearn import model_selection

In [5]:
x_trn, x_tst, y_trn, y_tst = model_selection.train_test_split(admit_x, admit_y, test_size=0.20, random_state=350)

# PCA

In [138]:
from sklearn.preprocessing import StandardScaler 

In [139]:
scaler = StandardScaler()

In [140]:
from sklearn.decomposition import PCA

In [141]:
admit_x_scale = scaler.fit_transform(admit_x)
x_trn_scale = scaler.fit_transform(x_trn)
x_tst_scale = scaler.fit_transform(x_tst)

C:\Users\Adam Chang\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Adam Chang\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\Adam Chang\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Adam Chang\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\Adam Chang\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWar

In [142]:
pca = PCA(n_components = 7)
pca.fit(admit_x_scale)

for i in range(7):
    print("Component %d" % (i+1))
    print("Explained Variance %.2f" % pca.explained_variance_[i])
    print(pca.components_[i])
    plt.clf()
    c_scalar = np.matmul(admit_x_scale, pca.components_[i])
    plt.xlabel("Projection onto Component %d" % (i+1))
    plt.ylabel("Admit Prob.")
    plt.plot(c_scalar, admit_y, 'o')
    plt.savefig("images/proj_component_%d.png" % (i+1))

Component 1
Explained Variance 4.74
[-0.40361694 -0.40120339 -0.3834133  -0.38466158 -0.3471675  -0.42103736
 -0.28872336]
Component 2
Explained Variance 0.74
[-0.27452263 -0.11081202  0.2497745   0.3434338   0.42602733 -0.01504455
 -0.74182095]
Component 3
Explained Variance 0.56
[-0.36263743 -0.46108277  0.09291061  0.17312554  0.46450082 -0.24073704
  0.58615408]
Component 4
Explained Variance 0.38
[-0.14482269 -0.12684268  0.64173368  0.32685429 -0.64859464 -0.13665843
  0.07022078]
Component 5
Explained Variance 0.26
[ 0.04779188 -0.03008301  0.60698049 -0.76276584  0.21111694 -0.01945709
 -0.04031081]
Component 6
Explained Variance 0.18
[-0.25090997  0.72652707  0.03207901  0.01981706  0.09562054 -0.62726591
  0.07193756]
Component 7
Explained Variance 0.15
[ 0.73757122 -0.2632937   0.02642093  0.12279282  0.08846344 -0.59332137
 -0.10503789]


In [150]:
from sklearn.linear_model import LinearRegression, LogisticRegression

In [151]:
# Using linear regression on PCA (1-7 components)

for i in range(7):
    print("Using %d components:" % (i+1))
    
    pca = PCA(n_components = i+1)
    x_trn_pca = pca.fit_transform(x_trn_scale)
    x_tst_pca = pca.transform(x_tst_scale)

    lr = LinearRegression()
    lr.fit(x_trn_pca, y_trn)
    print("Coef: %s" % lr.coef_)
    print("Intercept %s" % lr.intercept_)
    print("R^2: %.2f" % lr.score(x_trn_pca, y_trn))
    print("Test MSE: %.4f" % np.mean(np.square(y_tst - lr.predict(x_tst_pca))))
    print(np.matmul(lr.coef_, pca.components_))
    print()

Using 1 components:
Coef: [-0.05576297]
Intercept 0.727125
R^2: 0.78
Test MSE: 0.0062
[0.02248775 0.02244954 0.02155522 0.02174614 0.01929519 0.02358981
 0.01528144]

Using 2 components:
Coef: [-0.05576297 -0.00414248]
Intercept 0.727125
R^2: 0.78
Test MSE: 0.0060
[0.02350679 0.02265821 0.02069285 0.02048776 0.01770937 0.02349438
 0.01863212]

Using 3 components:
Coef: [-0.05576297 -0.00414248 -0.01550427]
Intercept 0.727125
R^2: 0.79
Test MSE: 0.0058
[0.02980593 0.02976151 0.01850598 0.01729205 0.0098336  0.02713291
 0.01088714]

Using 4 components:
Coef: [-0.05576297 -0.00414248 -0.01550427 -0.01768428]
Intercept 0.727125
R^2: 0.80
Test MSE: 0.0056
[0.03209665 0.03226206 0.00720525 0.01119189 0.02123653 0.02957133
 0.01030158]

Using 5 components:
Coef: [-0.05576297 -0.00414248 -0.01550427 -0.01768428  0.00397417]
Intercept 0.727125
R^2: 0.80
Test MSE: 0.0056
[0.03254699 0.03214769 0.0096451  0.00823477 0.02207263 0.02921066
 0.01007454]

Using 6 components:
Coef: [-0.05576297 -0.004

## Classficiation

In [152]:
admit_y_cls = (admit_y > np.median(admit_y)) * 1
y_trn_cls = (y_trn > np.median(admit_y)) * 1
y_tst_cls = (y_tst > np.median(admit_y)) * 1

In [153]:
# Using logistic regression on PCA (1-7 components)

for i in range(7):
    print("Using %d components:" % (i+1))
    
    pca = PCA(n_components = i+1)
    x_trn_pca = pca.fit_transform(x_trn_scale)
    x_tst_pca = pca.transform(x_tst_scale)

    logr = LogisticRegression()
    logr.fit(x_trn_pca, y_trn_cls)
    print("Coef: %s" % logr.coef_)
    print("Test Accuracy: %.4f" % (np.sum(y_tst_cls == logr.predict(x_tst_pca)) / len(y_tst_cls)))
    print()

Using 1 components:
Coef: [[-1.61256334]]
Test Accuracy: 0.8900

Using 2 components:
Coef: [[-1.60772479 -0.3037459 ]]
Test Accuracy: 0.8600

Using 3 components:
Coef: [[-1.62694271 -0.31067103 -0.22893158]]
Test Accuracy: 0.8800

Using 4 components:
Coef: [[-1.63517069 -0.31521748 -0.23285306 -0.28575366]]
Test Accuracy: 0.8800

Using 5 components:
Coef: [[-1.63416499 -0.31589112 -0.23275488 -0.28438636 -0.06506967]]
Test Accuracy: 0.8700

Using 6 components:
Coef: [[-1.67174558 -0.34744008 -0.25612129 -0.2024366  -0.13403168 -1.03345872]]
Test Accuracy: 0.8700

Using 7 components:
Coef: [[-1.68665131 -0.35401632 -0.24543345 -0.18486214 -0.13878989 -1.03612143
  -0.39717803]]
Test Accuracy: 0.8800



C:\Users\Adam Chang\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Adam Chang\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Adam Chang\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Adam Chang\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Adam Chang\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Spec

In [161]:
pca = PCA(n_components = 2)
admit_x_pca = pca.fit_transform(admit_x_scale)
plt.plot(admit_x_pca[admit_y > np.median(admit_y)][:,0], admit_x_pca[admit_y > np.median(admit_y)][:,1], 'o', label = 'Top')
plt.plot(admit_x_pca[admit_y <= np.median(admit_y)][:,0], admit_x_pca[admit_y <= np.median(admit_y)][:,1], 'o', label = 'Bottom')
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.legend()
plt.savefig("images/c1c2.png")
plt.clf()

In [162]:
pca = PCA(n_components = 3)
admit_x_pca = pca.fit_transform(admit_x_scale)
plt.plot(admit_x_pca[admit_y > np.median(admit_y)][:,0], admit_x_pca[admit_y > np.median(admit_y)][:,2], 'o', label = 'Top')
plt.plot(admit_x_pca[admit_y <= np.median(admit_y)][:,0], admit_x_pca[admit_y <= np.median(admit_y)][:,2], 'o', label = 'Bottom')
plt.xlabel("Component 1")
plt.ylabel("Component 3")
plt.legend()
plt.savefig("images/c1c3.png")
plt.clf()

In [163]:
pca = PCA(n_components = 3)
admit_x_pca = pca.fit_transform(admit_x_scale)
plt.plot(admit_x_pca[admit_y > np.median(admit_y)][:,1], admit_x_pca[admit_y > np.median(admit_y)][:,2], 'o', label = 'Top')
plt.plot(admit_x_pca[admit_y <= np.median(admit_y)][:,1], admit_x_pca[admit_y <= np.median(admit_y)][:,2], 'o', label = 'Bottom')
plt.xlabel("Component 2")
plt.ylabel("Component 3")
plt.legend()
plt.savefig("images/c2c3.png")
plt.clf()

# K-Means

In [164]:
from sklearn.cluster import KMeans

In [166]:
for nc in [2, 3, 6, 10]:
    km = KMeans(n_clusters = nc)
    km.fit(x_trn_scale)
    
    for i in range(nc):
        cluster_y = y_trn[km.labels_ == i]
        plt.clf()
        plt.title("K-Means (n = %d) - Cluster %d" % (nc, i))
        plt.hist(cluster_y)
        plt.savefig("images/km_n%d_c%d.png" % (nc, i))
    print("==========================================================")

In [167]:
from sklearn.model_selection import KFold

In [168]:
for nc in range(1,21):
    km = KMeans(n_clusters = nc)
    mses = []
    
    kf = KFold(n_splits=5, random_state=350)
    
    for train_index, test_index in kf.split(x_trn_scale):
        km.fit(x_trn_scale[train_index])

        km_pred = km.predict(x_trn_scale[test_index])
        y_pred = np.zeros(len(test_index))        

        for i in range(len(test_index)):
            y_pred[i] = np.mean(y_trn[train_index][km.labels_ == km_pred[i]])
            
        mses.append(np.mean(np.square(y_trn[test_index] - y_pred)))

    print("%d Clusters MSE: %0.4f" % (nc, np.mean(mses)))

1 Clusters MSE: 0.0190
2 Clusters MSE: 0.0190
3 Clusters MSE: 0.0192
4 Clusters MSE: 0.0192
5 Clusters MSE: 0.0194
6 Clusters MSE: 0.0198
7 Clusters MSE: 0.0197
8 Clusters MSE: 0.0196
9 Clusters MSE: 0.0197
10 Clusters MSE: 0.0198
11 Clusters MSE: 0.0197
12 Clusters MSE: 0.0200
13 Clusters MSE: 0.0199
14 Clusters MSE: 0.0201
15 Clusters MSE: 0.0201
16 Clusters MSE: 0.0201
17 Clusters MSE: 0.0211
18 Clusters MSE: 0.0207
19 Clusters MSE: 0.0204
20 Clusters MSE: 0.0204


In [169]:
km = KMeans(n_clusters = 6)
km.fit(x_trn_scale)

km_pred = km.predict(x_tst_scale)
y_pred = np.zeros(len(y_tst))

for i in range(len(y_tst)):
    y_pred[i] = np.mean(y_trn[km.labels_ == km_pred[i]])
            
print(np.mean(np.square(y_tst - y_pred)))

0.0075672731548239325


In [171]:
from scipy import stats

In [173]:
km = KMeans(n_clusters = 6)
km.fit(x_trn_scale)

km_pred = km.predict(x_tst_scale)
y_pred = np.zeros(len(y_tst_cls))

for i in range(len(y_tst)):
    y_pred[i] = stats.mode(y_trn_cls[km.labels_ == km_pred[i]])[0]
            
print(np.sum(y_pred == y_tst_cls) / len(y_tst_cls))

0.9
